In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Read datasets

In [1]:
# This file tells to which set (prior, train, test) an order belongs.
# You are predicting reordered items only for the test set orders. 'order_dow' is the day of week.

# COLUMNS:
# order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
orders = pd.read_csv('./data/orders.csv',index_col = 'order_id')

NameError: name 'pd' is not defined

In [ ]:
orders.head()

In [ ]:
# These files specify which products were purchased in each order.
# order_products__prior.csv contains previous order contents for all customers.
# 'reordered' indicates that the customer has a previous order that contains the product. 
# Note that some orders will have no reordered items.
# You may predict an explicit 'None' value for orders with no reordered items.
# See the evaluation page for full details.

# COLUMNS
# order_id,product_id,add_to_cart_order,reordered 
train = pd.read_csv('./data/order_products__train.csv')
prior = pd.read_csv('./data/order_products__prior.csv')

Additional information

Пока не ясно, как эта информация может быть использована.

In [ ]:
products    = pd.read_csv('./data/products.csv',index_col='product_id')
aisles      = pd.read_csv('./data/aisles.csv',index_col = 'aisle_id')
departments = pd.read_csv('./data/departments.csv', index_col = 'department_id')

В целом, что можно делать?

Ответ на какой вопрос:

Сгруппировать покупки по пользователю и по заказу

В тестовых заказах известна часть продуктов. Надо указать, какие ещё продукты будут дозаказаны.
Можно для каждого пользователя и продукта обучить классификатор, который будет отвечать на вопрос - купить ли пользователь это? 
Итого: на выходе требуется указать id продуктов, которые будут куплены.
Итого, это может быть классификатор. Либо глубокая нейронная сеть.

Для чего нужны проходы? Наверное, покупка одного продукта из прохода увеличивает вероятность покупки продукта из того же прохода.

Baseline решение:
найти наиболее частые группы товаров и проверить точность решения. Для этого можно использовать биграммы.
Для начала попробуем сгруппировать продукты по пользователям и товарам.

Попробовать объединить train и preorder в один сет. Добавить поля название продукта, пользователь, order_id, eval_set, order_number etc из таблицы orders. Сгруппировать по пользователям (первые уровень) и заказам (второй уровень).

In [ ]:
def addFeatures(df):
    priorSet = df.merge(products, left_on='product_id', right_index=True, how = 'left', copy = False)
    priorSet = priorSet.merge(aisles, left_on = 'aisle_id',right_index = True, how = 'left', copy = False)
    priorSet = priorSet.merge(departments, left_on = 'department_id', right_index = True, how = 'left', copy = False)
    priorSet = priorSet.merge(orders, left_on = 'order_id', right_index = True, how = 'left', copy = False)
    return priorSet

Как реализовать скрипт?

<b> Baseline решение - самые популярные продукты из его корзины </b>

Для каждого пользователя считаем:

1) <u>средний размер корзины</u>
Для этого группируем таблицу prior по 1) пользователям; 2) заказам. 
Исключаем train заказы
Вычисляем средний размер (GroupBy object, .size().mean() )
@TODO - проверить точность модели для разных способов округления среднего размера корзины;

2) <u>самые популярные продукты</u>
Группируем таблицу prior по 1) пользователям; 2) продуктам. Считаем статистику по продуктам, сортируем.

In [ ]:
def basketSize(userGroup):
    g = userGroup.groupby(['order_id'])
    return g['order_id'].size().mean()

In [ ]:
def save2File(fName,df):
    f = open(fName,'w')
    f.write('order_id,products\n')
    np.set_printoptions(linewidth = 500)
    for key, val in df.iteritems():
        k = np.array2string(val[1])[1:-1]+'\n'
        s = str(val[0])+','+k
        f.write(s)
    f.close()
    np.set_printoptions(linewidth = 75)

In [ ]:
testUserId = orders.query("eval_set == 'test'").user_id.values
testPriorOrderId = orders.query("(eval_set =='prior') and user_id in @testUsers").index.values
testPriorSet = prior.query('order_id in @testPriorOrderId')

In [ ]:
targetGroup = 'train'

userId = orders.query("eval_set == @targetGroup").user_id.values
priorOrderId = orders.query("(eval_set =='prior') and user_id in @userId").index.values
priorSet = prior.query('order_id in @priorOrderId')
#trainPriorSet = addFeatures(trainPriorSet)
priorSet = priorSet.merge(orders.loc[:,['user_id','order_number']],left_on='order_id',right_index=True,copy= False)

# calculate basket size
bs = priorSet.groupby('user_id').aggregate(basketSize)['order_id']
bs.rename('basket_size',inplace=True);

# plot basket size
bs.hist(bins = 70);
plt.xlabel('Mean basket size')
plt.ylabel('Users')

# calculate most popular goods
prod = priorSet.groupby(['user_id','product_id']).order_id.size().to_frame(name = 'count')
prod = prod.reset_index(level=1)
prod = prod.reset_index(level=0)

# reindex orders dataframe
c = orders.query("(user_id in @userId) and (eval_set == @targetGroup)").reset_index().set_index('user_id')
# create final set
res = dict()
for user_id, basket in prod.groupby('user_id'):
    res.update({user_id: (c.loc[user_id].order_id, 
                basket.sort_values('count',ascending=False).head(int(np.ceil(bs.get(user_id)))).product_id.values)})

In [ ]:
resTrain = res

In [ ]:
targetGroup = 'test'

userId = orders.query("eval_set == @targetGroup").user_id.values
priorOrderId = orders.query("(eval_set =='prior') and user_id in @userId").index.values
priorSet = prior.query('order_id in @priorOrderId')
#trainPriorSet = addFeatures(trainPriorSet)
priorSet = priorSet.merge(orders.loc[:,['user_id','order_number']],left_on='order_id',right_index=True,copy= False)

# calculate basket size
bs = priorSet.groupby('user_id').aggregate(basketSize)['order_id']
bs.rename('basket_size',inplace=True);

# plot basket size
bs.hist(bins = 70);
plt.xlabel('Mean basket size')
plt.ylabel('Users')

# calculate most popular goods
prod = priorSet.groupby(['user_id','product_id']).order_id.size().to_frame(name = 'count')
prod = prod.reset_index(level=1)
prod = prod.reset_index(level=0)

# reindex orders dataframe
c = orders.query("(user_id in @userId) and (eval_set == @targetGroup)").reset_index().set_index('user_id')

In [ ]:
# create final set
res = dict()
for user_id, basket in prod.groupby('user_id'):
    basket = basket.sort_values('count',ascending=False).assign(popularity = basket['count']/basket['count'].sum())
    top = basket.head(int(np.round(bs.get(user_id))))
    top = top[top.popularity > 0.01]
    res.update({user_id: (c.loc[user_id].order_id, top.product_id.values)})
    
save2File('solution5.csv',res)

In [ ]:
def f1Score(y_true, y_pred):
    fullSet      = np.unique(np.hstack((y_pred,y_true)))
    intersection = np.intersect1d(y_true,y_pred).size
    precession   = intersection/float(y_pred.shape[0])
    recall       = intersection/float(y_true.shape[0])
    return [precession, recall]

In [ ]:
for order in resTrain:
    